In [1]:
import sys


sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid.services.store_registry import StoreRegistry
from sem_covid import config
from sem_covid.adapters.data_source import BinaryDataSource
from sem_covid.services.ml_pipelines.pwdb_classifiers_pipeline import FeatureEngineering, EMBEDDING_COLUMN, \
    TEXTUAL_DATA, ModelTraining

In [2]:
import tensorflow_hub as hub

In [3]:
from sklearn.metrics.pairwise import pairwise_distances
from sem_covid import config
from sem_covid.services.store_registry import StoreRegistry
from sem_covid.services.data_registry import Dataset

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
import plotly.express as px
import concurrent.futures

In [83]:
from transformers import AutoTokenizer, AutoModel

In [84]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-eurlex")
model = AutoModel.from_pretrained("nlpaueb/bert-base-uncased-eurlex")

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/bert-base-uncased-eurlex were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [113]:
inputs = tokenizer('Hello', return_tensors='pt')

In [115]:
output = model(**inputs)

In [105]:
import numpy as np

In [205]:
from scipy.spatial.distance import cosine, euclidean

In [153]:
def cosine_sim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [172]:
def encode_text(text: str, model, tokenizer):
    return model(**tokenizer(text, return_tensors='pt'))['pooler_output'].detach().numpy()[0]

In [228]:
h1 = encode_text('Tomorrow the judge will do the hearing', model, tokenizer)
h1[:5]

array([-0.1866121,  0.6182396,  0.9996494, -0.5832228, -0.9880023],
      dtype=float32)

In [229]:
h2 = encode_text('the court proceeding will place tomorrow', model, tokenizer)
h2[:5]

array([-0.16304274,  0.56621265,  0.9995914 , -0.50024855, -0.98523676],
      dtype=float32)

In [230]:
cosine_sim(h1, h2)

0.99654305

In [199]:
1- cosine(h1,h2)

0.9426586627960205

In [231]:
1/(1+euclidean(h1,h2))


0.38564461516762555

In [4]:
class FeatureEngineeringBERT(FeatureEngineering):

    def __init__(self, feature_store_name: str):
        super().__init__(feature_store_name)
        self.model = None

    def load_language_model(self):
        model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
        self.model = hub.load(model_url)

    def compute_document_embeddings(self):
        assert self.model is not None
        self.df[EMBEDDING_COLUMN] = pd.Series(self.model(self.df[TEXTUAL_DATA]).numpy().tolist(), index=self.df.index)

In [5]:
class PWDBClassifiersBERT:
    """
        This class aims to unify the feature engineering pipeline and the model training pipeline.
    """

    @classmethod
    def feature_engineering(cls):
        """
            This method executes feature engineering pipeline with preset parameters.
        :return:
        """
        worker = FeatureEngineeringBERT(feature_store_name='fs_pwdb_bert')
        worker.execute()

    @classmethod
    def model_training(cls):
        """
            This method executes training model pipeline with preset parameters.
        :return:
        """
        worker = ModelTraining(feature_store_name='fs_pwdb_bert',
                               experiment_name="PyCaret_pwdb_bert")
        worker.execute()


In [ ]:
PWDBClassifiersBERT.model_training()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.7159,0.0000,0.6132,0.7007,0.6873,0.6976,0.7004,0.0910
lightgbm,Light Gradient Boosting Machine,0.7103,0.0000,0.5609,0.6569,0.6645,0.6900,0.6933,20.0050
lr,Logistic Regression,0.7071,0.0000,0.5365,0.6479,0.6505,0.6862,0.6902,0.1330
ridge,Ridge Classifier,0.7004,0.0000,0.5309,0.6327,0.6372,0.6786,0.6834,0.0260
nb,Naive Bayes,0.6782,0.0000,0.5523,0.6469,0.6426,0.6573,0.6605,0.0260
knn,K Neighbors Classifier,0.6738,0.0000,0.5582,0.6468,0.6324,0.6521,0.6554,0.0700
et,Extra Trees Classifier,0.6616,0.0000,0.4932,0.6045,0.6066,0.6353,0.6414,0.4290
rf,Random Forest Classifier,0.6316,0.0000,0.4503,0.5698,0.5696,0.6014,0.6085,0.3700
gbc,Gradient Boosting Classifier,0.6282,0.0000,0.4394,0.6073,0.5963,0.6027,0.6055,62.4700
lda,Linear Discriminant Analysis,0.6282,0.0000,0.4947,0.5911,0.5896,0.6039,0.6068,0.1040


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:59:30
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  2.5min


In [89]:
tmp_pipeline = FeatureEngineeringBERT(feature_store_name='fs_pwdb_bert')

In [90]:
tmp_pipeline.execute()


N/A% (0 of 1288) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

In [16]:
fs_store = StoreRegistry.es_feature_store()

In [95]:
tmp_df = fs_store.get_features('fs_pwdb_bert_x')

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [26]:
import pandas as pd

In [50]:
pd.DataFrame(list(tmp_df.loc['0'].values))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,-0.028826,-0.047585,0.042769,0.04712,0.06131,-0.053181,0.044774,-0.038114,0.052925,-0.06348,-0.045075,-0.024508,-0.012668,0.030686,-0.015553,-0.048582,0.034158,-0.047688,-0.057118,0.064871,0.020726,0.049543,0.040587,-0.06176,-0.020442,0.024156,-0.063852,0.005734,-0.038549,-0.023745,0.053987,0.005365,0.05883,0.036171,-0.032244,-0.061297,-0.030145,-0.000025,0.044332,0.034034,-0.01923,0.052538,-0.03114,0.048284,-0.032084,-0.041114,-0.049541,-0.055734,-0.005414,-0.045223,-0.060587,0.04021,0.013222,0.007668,0.014207,-0.062315,-0.057207,-0.062599,0.062472,-0.065007,0.056621,0.032553,-0.064813,0.020712,-0.046287,-0.027282,0.052268,-0.007141,-0.040662,-0.050776,0.048954,-0.017708,0.016251,0.02407,-0.024363,-0.065451,-0.034397,-0.037271,-0.048734,0.010097,0.016321,0.012018,-0.004343,-0.05251,0.019063,0.043017,-0.032928,0.064691,-0.033416,-0.063521,-0.036796,-0.045641,-0.057194,-0.052032,0.000821,0.012738,0.057615,0.063388,0.036389,-0.064652,-0.048936,-0.024082,0.003151,0.048055,-0.064909,0.030462,0.048566,0.027044,0.001678,0.059067,0.002348,0.006676,0.027932,0.014102,0.028787,0.062345,-0.064109,0.064187,-0.026606,0.0438,0.051729,0.02442,0.042106,-0.007512,0.054382,-0.051818,0.033186,0.044009,0.035971,0.019952,-0.065376,-0.059918,0.064041,-0.01833,0.025722,0.005355,0.021969,0.0174,-0.06187,0.020787,-0.060784,0.065448,-0.05266,0.002762,-0.043721,-0.005837,0.049297,-0.062706,0.021121,-0.001965,-0.06,0.000583,-0.043938,0.001723,0.007441,0.002614,0.026105,0.050119,-0.004238,-0.055984,-0.061916,0.033391,0.031578,0.058209,0.059823,0.00429,0.064801,0.048062,0.051004,-0.034206,0.061849,-0.004981,0.031862,0.048749,-0.054924,-0.022816,-0.048697,-0.052327,-0.064906,-0.064273,0.055151,0.056606,-0.018196,-0.006035,0.026019,-0.016108,-0.050609,0.017508,-0.048372,-0.048248,-0.049511,-0.046592,-0.037415,-0.018717,0.011999,-0.021892,-0.056029,0.021819,-0.010902,0.050137,-0.063975,-0.023715,0.027952,0.042166,0.045822,-0.032007,0.05799,0.060643,0.044399,-0.049194,-0.037926,0.065441,-0.006702,0.063078,-0.01248,-0.049296,0.040632,0.050567,0.020406,0.045083,-0.004264,-0.012815,-0.047492,-0.061182,0

In [74]:
tmp_pipeline.df[EMBEDDING_COLUMN]

KeyError: 'embeddings'

In [86]:
tmp_pipeline.df[EMBEDDING_COLUMN] = pd.Series(tmp_pipeline.model(tmp_pipeline.df[TEXTUAL_DATA]).numpy().tolist(), index = tmp_pipeline.df.index)

In [96]:
tmp_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.028826,-0.047585,0.042769,0.047120,0.061310,-0.053181,0.044774,-0.038114,0.052925,-0.063480,-0.045075,-0.024508,-0.012668,0.030686,-0.015553,-0.048582,0.034158,-0.047688,-0.057118,0.064871,0.020726,0.049543,0.040587,-0.061760,-0.020442,0.024156,-0.063852,0.005734,-0.038549,-0.023745,0.053987,0.005365,0.058830,0.036171,-0.032244,-0.061297,-0.030145,-0.000025,0.044332,0.034033,-0.019230,0.052538,-0.031140,0.048284,-0.032084,-0.041114,-0.049541,-0.055734,-0.005414,-0.045222,-0.060587,0.040210,0.013222,0.007668,0.014207,-0.062315,-0.057207,-0.062599,0.062472,-0.065007,0.056621,0.032553,-0.064813,0.020712,-0.046287,-0.027282,0.052268,-0.007141,-0.040662,-0.050776,0.048954,-0.017708,0.016251,0.024070,-0.024363,-0.065451,-0.034397,-0.037271,-0.048734,0.010098,0.016321,0.012018,-0.004343,-0.052510,0.019063,0.043017,-0.032928,0.064691,-0.033416,-0.063521,-0.036796,-0.045641,-0.057194,-0.052032,0.000821,0.012738,0.057615,0.063388,0.036389,-0.064652,-0.048936,-0.024083,0.003151,0.048055,-0.064909,0.030462,0.048566,0.027044,0.001679,0.059067,0.002348,0.006676,0.027932,0.014102,0.028787,0.062345,-0.064109,0.064187,-0.026606,0.043800,0.051729,0.024420,0.042106,-0.007512,0.054382,-0.051819,0.033186,0.044009,0.035971,0.019952,-0.065376,-0.059918,0.064041,-0.018330,0.025722,0.005355,0.021969,0.017400,-0.061870,0.020787,-0.060784,0.065448,-0.052660,0.002762,-0.043721,-0.005837,0.049297,-0.062706,0.021121,-0.001965,-0.060000,0.000584,-0.043938,0.001723,0.007441,0.002614,0.026105,0.050119,-0.004238,-0.055984,-0.061916,0.033391,0.031578,0.058209,0.059823,0.004290,0.064801,0.048062,0.051004